# 🏥 FHIR Integration Prototype

This notebook simulates loading FHIR resources and converting them to a dataframe for ML model input.

In [ ]:

import json
import pandas as pd
import requests

# Example FHIR Patient and Observation sample (locally or from sandbox)
def fetch_fhir_observations(patient_id="example", server_url="https://hapi.fhir.org/baseR4"):
    url = f"{server_url}/Observation?subject=Patient/{patient_id}"
    r = requests.get(url, headers={"Accept": "application/fhir+json"})
    return r.json() if r.status_code == 200 else {}

data = fetch_fhir_observations()


In [ ]:

# Convert FHIR Observation bundle to structured dataframe
def fhir_observation_to_df(bundle):
    records = []
    for entry in bundle.get("entry", []):
        resource = entry["resource"]
        if resource["resourceType"] == "Observation":
            rec = {
                "code": resource["code"]["text"] if "text" in resource["code"] else resource["code"]["coding"][0]["code"],
                "value": resource.get("valueQuantity", {}).get("value", None),
                "unit": resource.get("valueQuantity", {}).get("unit", None),
                "date": resource.get("effectiveDateTime", None)
            }
            records.append(rec)
    return pd.DataFrame(records)

obs_df = fhir_observation_to_df(data)
obs_df.head()


In [ ]:

# Placeholder: simulate mapping to model-ready format
# (e.g., extract blood pressure, glucose, cholesterol, etc.)
# This depends on hospital-specific mappings
obs_pivot = obs_df.pivot(index="date", columns="code", values="value")
obs_pivot.head()


In [ ]:

# Future: integrate with trained model for prediction
# For demo, use dummy features
from xgboost import XGBClassifier
import numpy as np

# Simulated data
sample_input = pd.DataFrame([{
    'age': 55, 'totChol': 220, 'sysBP': 135, 'glucose': 100, 'male': 1, 'cigsPerDay': 5,
    'diaBP': 85, 'BPMeds': 0, 'prevalentStroke': 0, 'diabetes': 0, 'prevalentHyp': 1,
    'heartRate': 75, 'education': 2
}])

# Load saved model (in practice, you'd use booster from tuning)
model = XGBClassifier()
model.fit(sample_input, [0])  # Dummy fit for syntax

# Predict CHD risk
# risk = model.predict_proba(sample_input)[:, 1]
# print("Predicted CHD Risk:", risk)
print("🔧 FHIR integration structure ready — connect to real model next.")
